Intialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim, length

Loading Table

In [0]:
df_silver = spark.table("workspace.bronze.erp_loc_a101")
df_silver.display()

Exploring Data

In [0]:
df_silver.printSchema()
df_silver.summary().show()

In [0]:
df_silver.select("CNTRY").distinct().display()

In [0]:
%sql
SELECT *
FROM workspace.silver.crm_customers 
LIMIT 10;

Trimming Data

In [0]:
for field in df_silver.schema.fields:
    if isinstance(field.dataType, StringType):
        df_silver = df_silver.withColumn(field.name, trim(col(field.name)))

Cleaning Data

In [0]:

df_silver = df_silver.withColumn("cid", F.regexp_replace(col("cid"), "-", ""))

In [0]:

df_silver = df_silver.withColumn(
    "cntry",
    F.when(col("cntry") == "DE", "Germany")
     .when(col("cntry").isin("US", "USA"), "United States")
     .when((col("cntry") == "") | col("cntry").isNull(), "n/a")
     .otherwise(col("cntry"))
)

Renaming columns

In [0]:
RENAME_MAP = {
    "cid": "customer_number",
    "cntry": "country"
}
for old_name, new_name in RENAME_MAP.items():
    df_silver = df_silver.withColumnRenamed(old_name, new_name)

In [0]:
df_silver.limit(10).display()

In [0]:
df_silver.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")